## MODIS Unbalanced dataset - Model training and validation ver one:

This model will use data obtained from MODIS (one day only), NDWI, NDWI_ICE, DEM SLOPE and EUCLIDEAN_NORM to predict water and non-water features.

#### 0. Load required libraries, site dependant constants and utility functions.

In [28]:
from os import listdir
from os.path import isfile, join
from lightgbm.sklearn import LGBMClassifier
import numpy as np
import seaborn as sns
import datetime
from sklearn import metrics
import joblib
# EOLearn libraries:
from eolearn.core import EOTask, EOPatch, LinearWorkflow, LoadTask, SaveTask, FeatureType, EOExecutor
from eolearn.core import OverwritePermission

# Add to python path parent dictionary
import sys
sys.path.append("../../")

# load site dependant constants (HERE YOU CAN CHOOSE DIFFERENT LOCATION)
from aoi_sites import upe_promice_area as site

# load utility functions
from utils import io_functions as io_utils
from utils import plot_functions as plot_utils

#### 1. Load sampled eopatches:

In [3]:
eopatches_filepath = '../../data/EOPatches/LANDSAT_8/UPE_PROMICE/UTM_22N/2013_05-2013_10_sampled_50k_modis_new_dataset_ver_1/'

eopatches = []
list_of_available_patches = io_utils.get_list_of_eopatches(eopatches_filepath)
list_in_chunks = io_utils.chunkIt(list_of_available_patches, 1 ) # number of chunks- 1 bc all can go
for element in list_in_chunks:
    print('Doing now following eopatches:', element )
    execution_args = []
    for eopatch_name in element:
        eopatches.append(EOPatch.load(eopatches_filepath+eopatch_name, lazy_loading=True))    

Doing now following eopatches: ['2974_maxcc_0.05_x-3_y-135', '2975_maxcc_0.05_x-3_y-136', '2976_maxcc_0.05_x-3_y-137', '2977_maxcc_0.05_x-3_y-138', '2978_maxcc_0.05_x-3_y-139', '2979_maxcc_0.05_x-3_y-140', '3062_maxcc_0.05_x-4_y-133', '3063_maxcc_0.05_x-4_y-134', '3064_maxcc_0.05_x-4_y-135', '3065_maxcc_0.05_x-4_y-136', '3066_maxcc_0.05_x-4_y-137', '3067_maxcc_0.05_x-4_y-138', '3068_maxcc_0.05_x-4_y-139', '3069_maxcc_0.05_x-4_y-140', '3174_maxcc_0.05_x-5_y-132', '3175_maxcc_0.05_x-5_y-133', '3176_maxcc_0.05_x-5_y-134', '3177_maxcc_0.05_x-5_y-135', '3178_maxcc_0.05_x-5_y-136', '3179_maxcc_0.05_x-5_y-137', '3180_maxcc_0.05_x-5_y-138', '3181_maxcc_0.05_x-5_y-139', '3182_maxcc_0.05_x-5_y-140', '3306_maxcc_0.05_x-6_y-130', '3307_maxcc_0.05_x-6_y-131', '3308_maxcc_0.05_x-6_y-132', '3309_maxcc_0.05_x-6_y-133', '3310_maxcc_0.05_x-6_y-134', '3311_maxcc_0.05_x-6_y-135', '3312_maxcc_0.05_x-6_y-136', '3313_maxcc_0.05_x-6_y-137', '3314_maxcc_0.05_x-6_y-138', '3315_maxcc_0.05_x-6_y-139', '3316_maxcc

#### 2. 80 /20 cross validation

In [4]:
len(eopatches)

45

In [5]:
import random
index_list = [i for i in range(0,45)]
random.shuffle(index_list)

no_of_chunks_to_test = 5
train_ID = []
test_ID = []
for x in range(5):
    test_ID.append(index_list.pop())
train_ID = index_list

In [6]:
print('Test indexes:', test_ID)
print('Train indexes:', train_ID)

Test indexes: [28, 16, 25, 27, 19]
Train indexes: [14, 18, 26, 31, 1, 44, 33, 34, 11, 41, 39, 20, 40, 9, 0, 37, 17, 35, 36, 8, 10, 4, 24, 5, 2, 13, 30, 38, 7, 6, 22, 42, 3, 32, 23, 15, 12, 43, 29, 21]


In [7]:
test_eopatches = []

for i in test_ID:
    test_eopatches.append( eopatches.pop(i))
train_eopatches = eopatches
del eopatches

####  3. Fetch and organise set

In [8]:
# Definition of the train and test patch IDs

# Set the features and the labels for train and test sets
features_train = np.array([eopatch.data['DATASET_CLD_200_dil_6_str2_SAMPLED'] for eopatch in train_eopatches if eopatch.data['DATASET_CLD_200_dil_6_str2_SAMPLED'].size > 0])
#features_train (number_of_patches,(time,width,height, bands))
labels_train = np.array([eopatch.mask['WATER_MASK_ST_025_SAMPLED'] for eopatch in train_eopatches if eopatch.mask['WATER_MASK_ST_025_SAMPLED'].size > 0])
#features_train (number_of_patches,(time,width,height, answer=True, False))
features_test = np.array([eopatch.data['DATASET_CLD_200_dil_6_str2_SAMPLED'] for eopatch in test_eopatches if eopatch.data['DATASET_CLD_200_dil_6_str2_SAMPLED'].size > 0 ])
labels_test = np.array([eopatch.mask['WATER_MASK_ST_025_SAMPLED'] for eopatch in test_eopatches if eopatch.mask['WATER_MASK_ST_025_SAMPLED'].size > 0])

In [9]:
#reshape to  (number_of_patches x time, width, height, bands)
features_train_stacked = np.vstack(features_train)
labels_train_stacked = np.vstack(labels_train)
features_test_stacked = np.vstack(features_test)
labels_test_stacked = np.vstack(labels_test)

# get shape
p_train_x_time, w, h, b = features_train_stacked.shape
p_test_x_time, w, h, b = features_test_stacked.shape


# reshape to n x m
#n - no of observation
#m - no of features, - bands in my case, misssing DEM

features_train = features_train_stacked.reshape(p_train_x_time * w * h, b)
labels_train = labels_train_stacked.reshape(p_train_x_time * w * h, 1)
features_test = features_test_stacked.reshape(p_test_x_time * w * h, b)
labels_test = labels_test_stacked.reshape(p_test_x_time * w * h, 1)


#@TODO
# remove points with no reference from training (so we dont train to recognize "no data")
# interpolation????
#mask_train = labels_train == 0
#features_train = features_train[~mask_train]
#labels_train = labels_train[~mask_train]

# remove points with no reference from test (so we dont validate on "no data", which doesn't make sense)
#mask_test = labels_test == 0
#features_test = features_test[~mask_test]
#labels_test = labels_test[~mask_test]


####  4. Data statistics

In [10]:
#to compare old data distribution (little less )

print('TRAIN DATA:')
tot_obs = np.size(labels_train)
print('Total observations: ', tot_obs )

tot_wat = np.count_nonzero(labels_train)
print('Total water count', tot_wat )

ratio_wat_vs_non_wat = tot_wat/ tot_obs
print('Percent of water feature among data:', ratio_wat_vs_non_wat )

print('\nTEST DATA:')

tot_obs = np.size(labels_test)
print('Total observations: ', tot_obs )

tot_wat = np.count_nonzero(labels_test)
print('Total water count', tot_wat )

ratio_wat_vs_non_wat = tot_wat/ tot_obs
print('Percent of water feature among data:', ratio_wat_vs_non_wat )
# dataset is half of the previous one!!!

TRAIN DATA:
Total observations:  9950000
Total water count 4218
Percent of water feature among data: 0.00042391959798994974

TEST DATA:
Total observations:  1550000
Total water count 233
Percent of water feature among data: 0.00015032258064516128


####  4. Model training

In [32]:
%%time

# Set up training classes
labels_unique = np.unique(labels_train)

# Set up the model
model = LGBMClassifier(boosting='gbdt',
    objective='binary',
    learning_rate=0.01,
    metric='binary_logloss', #mae: mean absolute error mse: mean squared error
    class_weight= 'balanced' 
                       
)

# train the model
model.fit(features_train, labels_train)


C:\Users\Kamil\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Kamil\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Wall time: 30.8 s


LGBMClassifier(boosting='gbdt', boosting_type='gbdt', class_weight='balanced',
               colsample_bytree=1.0, importance_type='split',
               learning_rate=0.01, max_depth=-1, metric='binary_logloss',
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective='binary',
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [29]:
# uncomment to save the model
model_base_name = 'NEW_model_WATER_NONWATER_MODIS_ver_1.0'
joblib.dump(model, './{}.pkl'.format(model_base_name))

['./NEW_model_WATER_NONWATER_MODIS_ver_1.0.pkl']

####  5. Model Validation

In [35]:
# predict the test labels
plabels_test = model.predict(features_test)

In [36]:
print('Classification accuracy {:.1f}%'.format(100 * metrics.accuracy_score(labels_test, plabels_test)))
print('Classification F1-score {:.1f}%'.format(100 * metrics.f1_score(labels_test, plabels_test, average='weighted')))

Classification accuracy 99.2%
Classification F1-score 99.6%


In [21]:
# predict the train labels
plabels_train = model.predict(features_train)

In [40]:
print('Classification accuracy {:.1f}%'.format(100 * metrics.accuracy_score(labels_train, plabels_train)))
print('Classification F1-score {:.1f}%'.format(100 * metrics.f1_score(labels_train, plabels_train, average='weighted')))

Classification accuracy 99.9%
Classification F1-score 99.9%


In [37]:
actual_water = np.count_nonzero(labels_test)
predicted_water = np.count_nonzero(plabels_test)

print('Water Test data size:',actual_water)

tp = np.count_nonzero(np.logical_and(labels_test.squeeze(), plabels_test.squeeze()))
print('Water True positive:', tp, 'and recall', 100* tp /actual_water, 'and precision:', tp /predicted_water)


Water Test data size: 233
Water True positive: 226 and recall 96.99570815450643 and precision: 0.018676142467564664


In [ ]:
# BECAUSE THE CLASS ARE NOT WEIGHTED, I HOPE SO

In [38]:
class_labels = np.unique(labels_test)
print(class_labels)
class_names = ['non-water', 'water']
class_names

[False  True]


['non-water', 'water']

In [39]:

f1_scores = metrics.f1_score(labels_test, plabels_test, labels=class_labels, average=None)
recall = metrics.recall_score(labels_test, plabels_test, labels=class_labels, average=None)
precision = metrics.precision_score(labels_test, plabels_test, labels=class_labels, average=None) 

print('             Class              =  F1  | Recall | Precision')
print('         --------------------------------------------------')
for idx, lulctype in enumerate([class_names[idx] for idx in class_labels]):
    print('         * {0:20s} = {1:2.1f} |  {2:2.1f}  | {3:2.1f}'.format(lulctype, 
                                                                         f1_scores[idx] * 100, 
                                                                         recall[idx] * 100, 
                                                                         precision[idx] * 100))

             Class              =  F1  | Recall | Precision
         --------------------------------------------------


C:\Users\Kamil\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  import sys


         * non-water            = 99.6 |  99.2  | 100.0
         * water                = 3.7 |  97.0  | 1.9


In [26]:
#### 6. Plot the standard and transposed Confusion Matrix: